In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

import anndata as ad
import scanpy as sc

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from umap import UMAP


from scroutines import basicu
from scroutines import powerplots
from scroutines.miscu import is_in_polygon

import utils_merfish
from merfish_datasets import merfish_datasets
from merfish_genesets import get_all_genesets

import importlib
importlib.reload(powerplots)
importlib.reload(utils_merfish)

In [ ]:
np.random.seed(0)

In [ ]:
def binning_pipe(adata, n=20, layer='lnorm', bin_type='depth_bin'):
    """
    """
    assert bin_type in ['depth_bin', 'width_bin']
    # bin it 
    depth_bins, depth_binned = utils_merfish.binning(adata.obs['depth'].values, n)
    width_bins, width_binned = utils_merfish.binning(adata.obs['width'].values, n)

    norm_ = pd.DataFrame(adata.layers[layer], columns=adata.var.index)
    norm_['depth_bin'] = depth_binned
    norm_['width_bin'] = width_binned
    
    norm_mean = norm_.groupby(bin_type).mean(numeric_only=True)
    norm_sem  = norm_.groupby(bin_type).sem(numeric_only=True)
    norm_std  = norm_.groupby(bin_type).std(numeric_only=True)
    norm_n    = norm_[bin_type].value_counts(sort=False)

    return norm_mean, norm_sem, norm_std, norm_n, depth_binned, width_binned, depth_bins, width_bins

In [ ]:
genesets = get_all_genesets()
genesets

In [ ]:
directories = merfish_datasets
print(merfish_datasets)

ddir = "/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/merfish/organized" 
fout = os.path.join(ddir, 'P28NRDR_v1l23_merged_240429.h5ad')
!ls $ddir/*240411.h5ad 

In [ ]:
%%time
names = [
    'P28NRb_ant', 
    'P28NRb_pos',
    'P28DRb_ant', 
    'P28DRb_pos',
]

alldata = {}
for name in names:
    alldata[name] = ad.read(os.path.join(ddir, f'{name}_ctxglut_240429.h5ad'))
    print(name, len(alldata[name]))

In [ ]:
for name, adata in alldata.items():
    print(name)
    gns = ['Scnn1a', 'Rorb', 'Igfbp4', 'Fos', 'Sorcs3']
    x = adata.obs['x']
    y = adata.obs['y']
    n = len(gns)

    fig, axs = plt.subplots(1,n,figsize=(n*6,1*5))
    for ax, gn in zip(axs, gns):
        g = adata[:,gn].layers['norm']
        utils_merfish.st_scatter_ax(fig, ax, x, y, gexp=g, s=1, title=gn)
    plt.show()

# viz

In [ ]:
v1l23_data = {}
for name in names:
    adata = alldata[name]
    # bound_l, bound_r, bound_d = adata.uns['bound_lrd']
    adatasub = adata[adata.obs['inside_v1l23']]
    v1l23_data[name] = adatasub

In [ ]:
gns = ['Scnn1a', 'Rorb', 'Igfbp4', 'Whrn', 'Fos', 'Cdh13', 'Sorcs3', 'Chrm2',]# 'Astn2', 'Fos']

name = 'P28DRb_pos' 
adata = alldata[name]
bound_l, bound_r, bound_d = adata.uns['bound_lrd']
x =  adata.obs['width']
y = -adata.obs['depth']
n = len(gns)

fig, axs = plt.subplots(n,1,figsize=(1*10,n*1.5))
fig.suptitle(name)
for ax, gn in zip(axs, gns):
    # gn = 'Scnn1a'
    g = np.log2(1+adata[:,gn].layers['norm']).reshape(-1,)
    
    vmax = np.percentile(g, 99)
    vmin = np.percentile(g,  5)
    cond = g > 1e-5
    
    utils_merfish.st_scatter_ax(fig, ax, x[~cond], y[~cond], gexp=g[~cond], s=2, title='', vmin=vmin, vmax=vmax, cmap='rocket_r') #, axis_off=False)
    utils_merfish.st_scatter_ax(fig, ax,  x[cond],  y[cond],  gexp=g[cond], s=2, title='', vmin=vmin, vmax=vmax, cmap='rocket_r') #, axis_off=False)
    
    ax.set_title(gn, loc='left', ha='right', y=0.5)
    ax.axhline(-bound_d, linestyle='--', linewidth=1, color='gray', zorder=2)
    ax.axvline( bound_l, linestyle='--', linewidth=1, color='gray', zorder=2)
    ax.axvline( bound_r, linestyle='--', linewidth=1, color='gray', zorder=2)
    # ax.grid(True)
    # ax.axis('on')
    
plt.show()

In [ ]:
gns = ['Fos', 'Cdh13', 'Sorcs3', 'Chrm2']
densities = []
for name in names:
    adata = v1l23_data[name]
    x = adata.obs['x']
    y = adata.obs['y']
    n = len(gns)
    width_range = adata.obs['width'].max() - adata.obs['width'].min()
    print(name, len(adata), width_range, len(adata)/width_range)
    densities.append(len(adata)/width_range)

    fig, axs = plt.subplots(1,n,figsize=(n*6,1*5))
    for ax, gn in zip(axs, gns):
        # gn = 'Scnn1a'
        g = np.log2(1+adata[:,gn].layers['norm'])
        utils_merfish.st_scatter_ax(fig, ax, x, y, gexp=g, s=5, title=gn)

    plt.show()

In [ ]:
gns = ['Cdh13', 'Sorcs3', 'Chrm2', 'Fos']
for name in names: 
    adatasub = v1l23_data[name]
    x =  adatasub.obs['width']
    y = -adatasub.obs['depth']
    n = len(gns)

    fig, axs = plt.subplots(n,1,figsize=(1*10,n*1.5))
    fig.suptitle(name)
    for ax, gn in zip(axs, gns):
        g = np.log2(1+adatasub[:,gn].layers['norm']).reshape(-1,)

        vmax = np.percentile(g, 99)
        vmin = np.percentile(g,  5)
        cond = g > 1e-5

        utils_merfish.st_scatter_ax(fig, ax, x[~cond], y[~cond], gexp=g[~cond], s=4, title='', vmin=vmin, vmax=vmax, cmap='rocket_r') #, axis_off=False)
        utils_merfish.st_scatter_ax(fig, ax,  x[cond],  y[cond],  gexp=g[cond], s=4, title='', vmin=vmin, vmax=vmax, cmap='rocket_r') #, axis_off=False)

        ax.set_title(gn, loc='left', ha='right', y=0.5)
        # ax.grid(True)
        # ax.axis('on')

    plt.show()

# do a clustering together
- abc genes

In [ ]:
agenes = genesets['a']
bgenes = genesets['b']
cgenes = genesets['c']
iegs   = genesets['i']
up_agenes = genesets['a_up']
abcgenes = np.hstack([agenes, bgenes, cgenes])
len(abcgenes)

In [ ]:
agenes_idx = basicu.get_index_from_array(adatasub.var.index.values, agenes)
bgenes_idx = basicu.get_index_from_array(adatasub.var.index.values, bgenes)
cgenes_idx = basicu.get_index_from_array(adatasub.var.index.values, cgenes)
igenes_idx = basicu.get_index_from_array(adatasub.var.index.values, iegs)

In [ ]:
# lnorm
for name in names:
    adatasub = v1l23_data[name]
    adatasub.layers['lnorm'] = np.log2(1+adatasub.layers['norm']) # .sum(axis=1)

In [ ]:
# cluster V1 L2/3 only
adata_merged = []
for i, name in enumerate(names):
    adatasub = v1l23_data[name].copy()
    if i == 0:
        genes = adatasub.var.index.values
    else:
        np.all(genes == adatasub.var.index.values)
        
    adatasub.obs.index = adatasub.obs.index + '_' + name
    adatasub.obs['sample'] = name
    adatasub.obs['depth_show'] = -adatasub.obs['depth'].values - i*500 # name
    adatasub.obs['width_show'] =  adatasub.obs['width'].values - np.min(adatasub.obs['width'].values)   # name
    adata_merged.append(adatasub)
    print(adatasub.shape)
    # break
    
adata_merged = ad.concat(adata_merged)
adata_merged_abcgenes = adata_merged[:,abcgenes].copy()

adata_merged, adata_merged_abcgenes

In [ ]:
adata = adata_merged

# PCA
pca = PCA(n_components=10)
pcs = pca.fit_transform(adata.layers['lnorm'])
ucs = UMAP(n_components=2, n_neighbors=30, random_state=0).fit_transform(pcs)

adata.obsm['pca'] = pcs
adata.obsm['umap'] = ucs
sc.pp.neighbors(adata, n_neighbors=30, use_rep='pca', random_state=0)



In [ ]:
# clustering
r = 0.4
sc.tl.leiden(adata, resolution=r, key_added=f'leiden_r{r}', random_state=0, n_iterations=1)

In [ ]:
# plot
gn = 'Slc17a7'
# gn = 'Fos'
# gn = 'Gad1'
g = np.log2(1+adata[:,gn].layers['norm'].reshape(-1,))

# add some quality metrics

fig, (ax1, ax2) = plt.subplots(1,2,figsize=(2*5,1*4))
utils_merfish.st_scatter_ax(fig, ax1, pcs[:,0], pcs[:,1], gexp=g)
utils_merfish.st_scatter_ax(fig, ax2, ucs[:,0], ucs[:,1], gexp=g)
plt.show()

# plot
gn = 'Slc17a7'
# gn = 'Fos'
# gn = 'Gad1'
g = adata[:,gn].layers['norm'].reshape(-1,)

# add some quality metrics

fig, (ax1, ax2) = plt.subplots(1,2,figsize=(2*5,1*4))
utils_merfish.st_scatter_ax(fig, ax1, pcs[:,0], pcs[:,1], gexp=g)
utils_merfish.st_scatter_ax(fig, ax2, ucs[:,0], ucs[:,1], gexp=g)
plt.show()

In [ ]:

clsts = adata.obs[f'leiden_r{r}'].astype(int)
xr =  adata.obs['width_show']
yr =  adata.obs['depth_show']
ux    = adata.obsm['umap'][:,0]
uy    = adata.obsm['umap'][:,1]
utils_merfish.plot_cluster(clsts, xr, yr, ux, uy, s=2)

samples, uniq_labels = pd.factorize(adata.obs['sample']) # .astype(int)
utils_merfish.plot_cluster(samples, xr, yr, ux, uy, s=2)

In [ ]:
clsts = adata.obs[f'leiden_r{r}'].astype(int)
uniq_clsts = np.unique(clsts)


for clst in uniq_clsts:
    show = (clsts == clst)
    xr =  adata.obs['width_show']
    yr =  adata.obs['depth_show']
    ux    = adata.obsm['umap'][:,0]
    uy    = adata.obsm['umap'][:,1]
    utils_merfish.plot_cluster(show, xr, yr, ux, uy, s=2, cmap=plt.cm.copper_r, suptitle=clst)

In [ ]:
clst_annots = [
     'Glu_nr_low', 
     'olig_and_low_qual',
     'Glu_dr_low', 
     'Glu_nr_high', 
     'Glu_dr_high', 
     'micro', 
     'astro',
     'GABA',
     'VLMC',
]
adata.uns['clst_annots'] = clst_annots

In [ ]:
adata.write(fout)

# figure out major cell population by marker genes; groups of genes; and quality metrics

In [ ]:
# plot
marker_genes = [
       'Ptprn', 'Slc17a7', 'Gad1', 'Fos', 
       
       'Gfap', 'Slc6a13', 'Slc47a1',
       'Grin2c', 'Aqp4', 'Rfx4', 'Sox21', 'Slc1a3',
       
       'Sox10', 'Pdgfra', 'Mog',
       
       'Pecam1', 'Cd34' , 'Tnfrsf12a', 'Sema3c', 
       'Zfhx3', 'Pag1', 'Slco2b1', 'Cx3cr1',
      ] 
gns = marker_genes
n = len(gns)
nx = 4
ny = int((n+nx-1)/nx)
# add some quality metrics
fig, axs = plt.subplots(ny,nx,figsize=(nx*5,ny*4))
for gn, ax in zip(gns, axs.flat):
    g = np.log2(1+adata[:,gn].layers['norm'].reshape(-1,))
    utils_merfish.st_scatter_ax(fig, ax, ucs[:,0], ucs[:,1], gexp=g)
    ax.set_title(gn)
plt.show()



In [ ]:
metrics = [
    'volume', 'anisotropy', 'perimeter_area_ratio', 'solidity', 
    'PolyT_raw', 'PolyT_high_pass', 'DAPI_raw', 'DAPI_high_pass', 
    'transcript_count', 'gncov', 'gnnum', 'fpcov', 
    'depth', 'width', 'sample' 
       ]
n = len(metrics)
nx = 5
ny = int((n+nx-1)/nx)
# add some quality metrics
fig, axs = plt.subplots(ny,nx,figsize=(nx*5,ny*4))
for metric, ax in zip(metrics, axs.flat):
    g = adata.obs[metric].values
    if metric == 'sample':
        g, uniq_lbls = pd.factorize(g)
    utils_merfish.st_scatter_ax(fig, ax, ucs[:,0], ucs[:,1], gexp=g, s=3)
    ax.set_title(metric)
plt.show()


In [ ]:
n = len(metrics)
nx = 5
ny = int((n+nx-1)/nx)
# add some quality metrics
fig, axs = plt.subplots(ny,nx,figsize=(nx*5,ny*4))
for metric, ax in zip(metrics, axs.flat):
    if metric == 'sample':
        g, uniq_lbls = pd.factorize(g)
    else:
        g = np.log10(1+adata.obs[metric].values)
    utils_merfish.st_scatter_ax(fig, ax, ucs[:,0], ucs[:,1], gexp=g, s=3)
    ax.set_title(metric)
plt.show()

# broad annotation and save

# visualize FISH

In [ ]:
# check
gns = ['Cdh13', 'Sorcs3', 'Chrm2', 'Fos']
x =  adata.obs['width_show']
y =  adata.obs['depth_show']
n = len(gns)

for gn in gns:
    fig, ax = plt.subplots(1,1,figsize=(1*10,1*3))
    g = adata[:,gn].layers['norm'].reshape(-1,)
    
    vmax = np.percentile(g, 99)
    vmin = np.percentile(g,  5)
    cond = g > 1e-5
    
    utils_merfish.st_scatter_ax(fig, ax, x[~cond], y[~cond], gexp=g[~cond], s=2, title='', vmin=vmin, vmax=vmax, cmap='rocket_r') #, axis_off=False)
    utils_merfish.st_scatter_ax(fig, ax,  x[cond],  y[cond],  gexp=g[cond], s=2, title='', vmin=vmin, vmax=vmax, cmap='rocket_r') #, axis_off=False)
    ax.set_title(gn, loc='left')
    
    # ax.grid(True)
    # ax.axis('on')
    
plt.show()

# stats

In [ ]:
stats = {}
for name in names:
    adatasub = v1l23_data[name]
    lnorm_mean, lnorm_sem, lnorm_std, n, d, w, db, wb = binning_pipe(adatasub, n=12)
    stats[name] = (lnorm_mean, lnorm_sem, lnorm_std, n, d, w, db, wb)
d.value_counts()

In [ ]:
# mean expression level across V1 L2/3 in NR
base_a0 = []
base_b0 = []
base_c0 = []
base_i0 = []
for name in ['P28NRb_ant', 'P28NRb_pos']:
    (lnorm_mean, lnorm_sem, lnorm_std, n, d, w, db, wb) = stats[name]
    base_a = np.mean(lnorm_mean.iloc[:,agenes_idx], axis=0) # across depth bins for each gene
    base_b = np.mean(lnorm_mean.iloc[:,bgenes_idx], axis=0) # across depth bins for each gene
    base_c = np.mean(lnorm_mean.iloc[:,cgenes_idx], axis=0) # across depth bins for each gene
    base_i = np.mean(lnorm_mean.iloc[:,igenes_idx], axis=0) # across depth bins for each gene
    
    base_a0.append(base_a)
    base_b0.append(base_b)
    base_c0.append(base_c)
    base_i0.append(base_i)
    
base_a0 = np.mean(base_a0, axis=0)
base_b0 = np.mean(base_b0, axis=0)
base_c0 = np.mean(base_c0, axis=0)
base_i0 = np.mean(base_i0, axis=0)

base_a0.shape, base_b0.shape, base_c0.shape, base_i0.shape

In [ ]:
means = {}
sems = {}
for name in names:
    (lnorm_mean, lnorm_sem, lnorm_std, n, d, w, db, wb) = stats[name]
    
    amean = np.mean(lnorm_mean.iloc[:,agenes_idx]-base_a0, axis=1) # a bin vector
    bmean = np.mean(lnorm_mean.iloc[:,bgenes_idx]-base_b0, axis=1) # a bin vector
    cmean = np.mean(lnorm_mean.iloc[:,cgenes_idx]-base_c0, axis=1) # a bin vector
    imean = np.mean(lnorm_mean.iloc[:,igenes_idx]-base_i0, axis=1) # a bin vector
    
    asem = np.mean(lnorm_sem.iloc[:,agenes_idx], axis=1)
    bsem = np.mean(lnorm_sem.iloc[:,bgenes_idx], axis=1)
    csem = np.mean(lnorm_sem.iloc[:,cgenes_idx], axis=1)
    isem = np.mean(lnorm_sem.iloc[:,igenes_idx], axis=1)
    
    means[name] = [amean, bmean, cmean, imean]
    sems[name] = [asem, bsem, csem, isem]
    

In [ ]:
for name in names:
    adatasub = v1l23_data[name]
    
    depth = adatasub.obs['depth'].values
    width = adatasub.obs['width'].values
    mat   = adatasub.layers['lnorm']

    fig, axs = plt.subplots(4,1,figsize=(10,4*1.5))
    fig.suptitle(name)
    gnames = ['A genes', 'B genes', 'C genes', 'IEGs']
    for i, (_idx, _baseline) in enumerate(zip([agenes_idx, bgenes_idx, cgenes_idx, igenes_idx], 
                                              [base_a0, base_b0, base_c0, base_i0])):
        ax = axs[i]
        lnorm_ = mat[:,_idx].copy()
        g = (lnorm_ - np.array(_baseline).reshape(1,-1)).mean(axis=1) 
        gn = gnames[i]
        if i < 3:
            vmax = +0.08 # np.percentile(g, 95)
            vmin = -0.08 # np.percentile(g,  0)
        else:
            vmax = +0.25 # np.percentile(g, 95)
            vmin = -0.25 # np.percentile(g,  0)

        utils_merfish.st_scatter_ax(fig, ax, width, -depth, gexp=g, s=5, title='', vmin=vmin, vmax=vmax, cmap='coolwarm') #, axis_off=False)
        ax.set_title(gn, loc='left', ha='right', y=0.5)
    plt.show()

In [ ]:
gnames = ['A genes (n=64)', 'B genes (n=35)', 'C genes (n=71)']

fig, axs = plt.subplots(1, 4, figsize=(5*4,4), sharex=True, sharey=True)

# ax.set_title('P28NR')
linestyle = '-'
for ax, name in zip(axs, ['P28NRb_ant', 'P28NRb_pos', 'P28DRb_ant', 'P28DRb_pos']):
    (lnorm_mean, lnorm_sem, lnorm_std, n, d, w, db, wb) = stats[name]
    amean, bmean, cmean, imean = means[name]
    asem, bsem, csem, isem = sems[name]
    
    x = np.mean(np.vstack([db[:-1], db[1:]]), axis=0)
    
    ax.plot(x, amean, label='A genes', color='C0', linestyle=linestyle)
    ax.fill_between(x, amean-asem, amean+asem, color='C0', alpha=0.1, edgecolor='none')
    ax.plot(x, bmean, label='B genes', color='C1', linestyle=linestyle)
    ax.fill_between(x, bmean-bsem, bmean+bsem, color='C1', alpha=0.1, edgecolor='none')
    ax.plot(x, cmean, label='C genes', color='C2', linestyle=linestyle)
    ax.fill_between(x, cmean-csem, cmean+csem, color='C2', alpha=0.1, edgecolor='none')
    ax.axhline(color='k', linestyle='--', zorder=1)

    sns.despine(ax=ax)
    ax.set_xticks([0, 100, 200, 300])
    ax.set_xlim(left=100, right=350)
    ax.set_ylim([-0.2, 0.2])
    ax.grid(False)
    ax.set_title(name)
axs[0].set_ylabel('mean (expr. +/- sem)')

# ax = axs[1]
# ax.set_title('P28DR')
# for ax, name, in zip(axs[2:], ['P28NR_ant', 'P28NR_pos'], ):
#     (lnorm_mean, lnorm_sem, lnorm_std, n, d, w, db, wb) = stats[name]
#     amean, bmean, cmean, imean = means[name]
#     asem, bsem, csem, isem = sems[name]
    
#     x = np.mean(np.vstack([db[:-1], db[1:]]), axis=0)
    
#     ax.plot(x, amean, label='A genes', color='C0', linestyle=linestyle)
#     ax.fill_between(x, amean-asem, amean+asem, color='C0', alpha=0.1, edgecolor='none')
#     ax.plot(x, bmean, label='B genes', color='C1', linestyle=linestyle)
#     ax.fill_between(x, bmean-bsem, bmean+bsem, color='C1', alpha=0.1, edgecolor='none')
#     ax.plot(x, cmean, label='C genes', color='C2', linestyle=linestyle)
#     ax.fill_between(x, cmean-csem, cmean+csem, color='C2', alpha=0.1, edgecolor='none')
#     ax.axhline(color='k', linestyle='--', zorder=1)

#     sns.despine(ax=ax)
#     ax.set_xticks([0, 100, 200, 300])
#     ax.set_xlim(left=100, right=350)
#     ax.set_ylim([-0.06, 0.06])
#     ax.grid(False)

    
fig.subplots_adjust(wspace=0.1)
# powerplots.savefig_autodate(fig, outdatadir+'/grant_saumya_lineq_abc_v3.pdf')

In [ ]:
gnames = ['Type A genes', 'Type B genes', 'Type C genes']

fig, axs = plt.subplots(1, 3, figsize=(5*3,4), sharex=True, sharey=True)

for i in range(3):
    ax = axs[i]
    sns.despine(ax=ax)
    ax.set_title(gnames[i])
    ax.grid(False)
    color = f'C{i}'
    
    for name in ['P28NRb_ant', 'P28NRb_pos']:
        (lnorm_mean, lnorm_sem, lnorm_std, n, d, w, db, wb) = stats[name]
        mean_vec = means[name][i]
        sem_vec = sems[name][i]

        x = np.mean(np.vstack([db[:-1], db[1:]]), axis=0)
        ax.plot(x, mean_vec, label='NR', color=color, linestyle='-')
        ax.fill_between(x, mean_vec-sem_vec, mean_vec+sem_vec, color=color, alpha=0.1, edgecolor='none')
    
    for name in ['P28DRb_ant', 'P28DRb_pos']:
        (lnorm_mean, lnorm_sem, lnorm_std, n, d, w, db, wb) = stats[name]
        mean_vec = means[name][i]
        sem_vec = sems[name][i]

        x = np.mean(np.vstack([db[:-1], db[1:]]), axis=0)
        ax.plot(x, mean_vec, label='DR', color=color, linestyle='--')
        ax.fill_between(x, mean_vec-sem_vec, mean_vec+sem_vec, color=color, alpha=0.1, edgecolor='none')
        
    ax.set_ylim([-0.2, 0.2])
    ax.axhline(color='k', linestyle='dotted', zorder=1)
    ax.set_xticks([100, 200, 300])
    ax.set_xlim(left=100, right=350)
    ax.set_xlabel('upper->lower cortical depth')
    
axs[0].set_ylabel('mean (expr. +/- sem)')
axs[2].legend(bbox_to_anchor=(1,1))

fig.subplots_adjust(wspace=0.1)
# powerplots.savefig_autodate(fig, outdatadir+'/grant_saumya_lineq_abc_v3.pdf')

In [ ]:
gnames = ['activity-dependent genes',]

fig, ax = plt.subplots(1, 1, figsize=(5*1,4))

i = 3
color = f'C4'

sns.despine(ax=ax)
ax.set_title(gnames[0])
ax.grid(False)

for name in ['P28NRb_ant', 'P28NRb_pos']:
    (lnorm_mean, lnorm_sem, lnorm_std, n, d, w, db, wb) = stats[name]
    mean_vec = means[name][i]
    sem_vec = sems[name][i]

    x = np.mean(np.vstack([db[:-1], db[1:]]), axis=0)
    ax.plot(x, mean_vec, label='NR', color=color, linestyle='-')
    ax.fill_between(x, mean_vec-sem_vec, mean_vec+sem_vec, color=color, alpha=0.1, edgecolor='none')

for name in ['P28DRb_ant', 'P28DRb_pos']:
    (lnorm_mean, lnorm_sem, lnorm_std, n, d, w, db, wb) = stats[name]
    mean_vec = means[name][i]
    sem_vec = sems[name][i]

    x = np.mean(np.vstack([db[:-1], db[1:]]), axis=0)
    ax.plot(x, mean_vec, label='DR', color=color, linestyle='--')
    ax.fill_between(x, mean_vec-sem_vec, mean_vec+sem_vec, color=color, alpha=0.1, edgecolor='none')

# ax.set_ylim([-0.08, 0.08])
ax.axhline(color='k', linestyle='dotted', zorder=1)
# ax.set_xticks([0, 100, 200, 300])
ax.set_xticks([100, 200, 300])
ax.set_xlim(left=100)
ax.set_xlabel('upper->lower cortical depth')
    
ax.set_ylabel('mean (expr. +/- sem)')
ax.legend(bbox_to_anchor=(1,1))

fig.subplots_adjust(wspace=0.1)
# powerplots.savefig_autodate(fig, outdatadir+'/grant_saumya_lineq_abc_v3.pdf')

# V1 vs outside of V1